# Imports

In [10]:
import numpy as np
import scipy

# Goal
This notebook looks at single qubit measurements. Given a $2 \times 2$ matrix representing the measurement operator, we return:
1. Eigenvalue/Eigenstate pairs
2. Measurement Probabilities as a functional on the quantum states

In [88]:
class MeasurementOperator:
    def __init__(self, density_matrix: np.array):
        self.density_matrix = density_matrix 
        self.eigenvalues, self.eigenstates = scipy.linalg.eig(density_matrix)
        
    def get_amplitudes(self, quantum_state: np.array) -> np.array:
        quantum_state = np.divide(quantum_state, scipy.linalg.norm(quantum_state))
        return self.eigenstates @ quantum_state @ self.eigenstates.T

    def get_probabilities(self, quantum_state: np.array) -> np.array:
        return self.get_amplitudes(quantum_state=quantum_state)**2


    def summary(self) -> None:
        pass

# Usage

We initialize the `MeasurementOperator` with the density matrix. After that, we can get the probabilities of each measurement using the get_probabilities method, which requires a quantum state as input

In [99]:
density_matrix = [[1, 0],[0, -1]]
mo = MeasurementOperator(density_matrix)

In [100]:
mo.get_probabilities([1,1])

array([0.5, 0.5])

# Example: Separating quantum states

We want to separate the quantum states $\ket{0}$ and $\frac{\ket{0}+\ket{1}}{\sqrt{2}}$. We do this by assembling:
- $E_1 := \frac{\sqrt{2}}{1+\sqrt{2}} \ket{1} \bra{1}$
- $E_2 := \frac{\sqrt{2}}{1+\sqrt{2}} \frac{(\ket{0} - \ket{1})(\bra{0} - \bra{1})}{2}$
- $E_3 := I - E_1 - E_2$

In [116]:
factor = np.divide(np.sqrt(2), 1+np.sqrt(2))
e_1 = factor * np.array([[0,0],[0,1]])
e_2 = factor * np.array([[1,-1],[-1,1]])
e_3 = np.array([[1,0],[0,1]]) - e_1 - e_2

We build the measurement matrix $M=E_1 + 2E_2 + 3E_3$

In [117]:
density_matrix = 1*e_1 + 2*e_2 + 3*e_3
mo = MeasurementOperator(density_matrix)
mo.density_matrix

array([[2.41421356, 0.58578644],
       [0.58578644, 1.24264069]])

We calculate the probabilities for the quantum state $\ket{0}$

In [118]:
mo.get_probabilities([1,0])

array([0.5, 0.5])

We calculate the probabilities for the quantum state $\frac{\ket{0} + \ket{1}}{2}$

In [119]:
mo.get_probabilities([1,1])

array([7.41362644e-35, 1.00000000e+00])